<a href="https://colab.research.google.com/github/santyxd3000/Undergraduate-Project/blob/main/Classifying_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PROYECTO DE GRADO - MODELOS DE CLASIFICACIÓN**

Santiago Pardo Morales - 202013025

In [12]:
#For using the GPU
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 438, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 438 (delta 121), reused 57 (delta 54), pack-reused 269
Receiving objects: 100% (438/438), 118.31 KiB | 11.83 MiB/s, done.
Resolving deltas: 100% (223/223), done.
***********************************************************************
We will now install RAPIDS via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/pip-install.py", line 14, in <module>
    for line in io.TextIOWrapper(output.stdout, encoding="utf-8"):
KeyboardInterrupt
^C


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display
import datetime
import scipy
sns.set()
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
import io
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn import model_selection
from sklearn.datasets import make_classification
from sklearn.model_selection import ParameterGrid
from sklearn.neighbors import LocalOutlierFactor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
#import cudf
#import cuml
#import cupy as cp
#from cuml import preprocessing as cpp
import tensorflow as tf
from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
physical_devices = tf.config.list_physical_devices('GPU')
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

In [5]:
#Leer datos etiquetados
df = pd.read_csv('df_with_labels.csv',sep=';').dropna()

In [6]:
df['Label'].unique()

array(['No Fault', 'Shading', 'Open Circuit', 'Unknown', 'Short Circuit'],
      dtype=object)

## **MÉTODO 1 - RED NEURONAL**

In [7]:
df['Label'].replace({'No Fault': 0, 'Shading': 1, 'Open Circuit': 2, 'Short Circuit': 3, 'Unknown':4}, inplace=True)

In [8]:
df[df['Label']==1].shape

(47256, 6)

In [9]:
x_data = df[['U_DC1','U_DC2','I_DC1','I_DC2']]
y_data = df[['Label']]

In [10]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x_data)
y_data = np.array(y_data)

In [34]:
#Divide data
x_train, x_temp, y_train, y_temp = train_test_split(x_scaled, y_data, train_size = 0.6, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, train_size = 0.5, random_state=42)

### Red Neuronal - Solo capa de entrada y de salida

In [35]:
ann_1 = Sequential()
ann_1.add(Dense(4, input_shape=(x_train.shape[1],),activation='relu'))

#output layer with softmax function
ann_1.add(Dense(5,activation='softmax'))

#compile model
ann_1.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [36]:
history= ann_1.fit(x_train,y_train, validation_data=(x_val,y_val),epochs=100, batch_size=32)
loss, accuracy= ann_1.evaluate(x_val,y_val)
print('Accuracy:', accuracy)
print('\nLoss:',loss)

Epoch 1/100
2820/2820 [==============================] - 12s 4ms/step - loss: 0.3189 - accuracy: 0.9439 - val_loss: 0.1056 - val_accuracy: 0.9806
Epoch 2/100
2820/2820 [==============================] - 11s 4ms/step - loss: 0.0822 - accuracy: 0.9823 - val_loss: 0.0721 - val_accuracy: 0.9819
Epoch 3/100
2820/2820 [==============================] - 11s 4ms/step - loss: 0.0626 - accuracy: 0.9835 - val_loss: 0.0609 - val_accuracy: 0.9826
Epoch 4/100
2820/2820 [==============================] - 9s 3ms/step - loss: 0.0527 - accuracy: 0.9844 - val_loss: 0.0525 - val_accuracy: 0.9839
Epoch 5/100
2820/2820 [==============================] - 8s 3ms/step - loss: 0.0434 - accuracy: 0.9858 - val_loss: 0.0441 - val_accuracy: 0.9857
Epoch 6/100
2820/2820 [==============================] - 11s 4ms/step - loss: 0.0370 - accuracy: 0.9876 - val_loss: 0.0401 - val_accuracy: 0.9887
Epoch 7/100
2820/2820 [==============================] - 9s 3ms/step - loss: 0.0341 - accuracy: 0.9903 - val_loss: 0.0387 - va

### Red Neuronal - Capa intermedia entre entrada y salida

In [39]:
ann_2 = Sequential()
ann_2.add(Dense(16, input_shape=(x_train.shape[1],),activation='relu'))
ann_2.add(Dense(4 ,activation='relu'))
#output layer with softmax function
ann_2.add(Dense(5,activation='softmax'))

#compile model
ann_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [40]:
history= ann_2.fit(x_train,y_train, validation_data=(x_val,y_val),epochs=100, batch_size=32)
loss, accuracy= ann_2.evaluate(x_val,y_val)
print('Accuracy:', accuracy)
print('\nLoss:',loss)

Epoch 1/100
2820/2820 [==============================] - 12s 4ms/step - loss: 0.1932 - accuracy: 0.9629 - val_loss: 0.0429 - val_accuracy: 0.9906
Epoch 2/100
2820/2820 [==============================] - 10s 4ms/step - loss: 0.0326 - accuracy: 0.9921 - val_loss: 0.0336 - val_accuracy: 0.9910
Epoch 3/100
2820/2820 [==============================] - 10s 4ms/step - loss: 0.0280 - accuracy: 0.9926 - val_loss: 0.0324 - val_accuracy: 0.9908
Epoch 4/100
2820/2820 [==============================] - 11s 4ms/step - loss: 0.0265 - accuracy: 0.9927 - val_loss: 0.0312 - val_accuracy: 0.9907
Epoch 5/100
2820/2820 [==============================] - 9s 3ms/step - loss: 0.0252 - accuracy: 0.9928 - val_loss: 0.0284 - val_accuracy: 0.9917
Epoch 6/100
2820/2820 [==============================] - 10s 4ms/step - loss: 0.0245 - accuracy: 0.9928 - val_loss: 0.0278 - val_accuracy: 0.9913
Epoch 7/100
2820/2820 [==============================] - 11s 4ms/step - loss: 0.0235 - accuracy: 0.9932 - val_loss: 0.0280 - 

### Red Neuronal - 2 capas intermedia entre entrada y salida

In [41]:
ann_3 = Sequential()
ann_3.add(Dense(64, input_shape=(x_train.shape[1],),activation='relu'))
ann_3.add(Dense(16 ,activation='relu'))
ann_3.add(Dense(4 ,activation='relu'))
#output layer with softmax function
ann_3.add(Dense(5,activation='softmax'))

#compile model
ann_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history= ann_3.fit(x_train,y_train, validation_data=(x_val,y_val),epochs=100, batch_size=32)
loss, accuracy= ann_3.evaluate(x_val,y_val)
print('Accuracy:', accuracy)
print('\nLoss:',loss)

### Red Neuronal - 3 capas intermedia entre entrada y salida

In [ ]:
ann_4 = Sequential()
ann_4.add(Dense(256, input_shape=(x_train.shape[1],),activation='relu'))
ann_4.add(Dense(64 ,activation='relu'))
ann_4.add(Dense(16 ,activation='relu'))
ann_4.add(Dense(4 ,activation='relu'))
#output layer with softmax function
ann_4.add(Dense(5,activation='softmax'))

#compile model
ann_4.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history= ann_4.fit(x_train,y_train, validation_data=(x_val,y_val),epochs=100, batch_size=32)
loss, accuracy= ann_4.evaluate(x_val,y_val)
print('Accuracy:', accuracy)
print('\nLoss:',loss)

### Red Neuronal - 4 capas intermedia entre entrada y salida

In [ ]:
ann_5 = Sequential()
ann_5.add(Dense(256, input_shape=(x_train.shape[1],),activation='relu'))
ann_5.add(Dense(64 ,activation='relu'))
ann_5.add(Dense(16 ,activation='relu'))
ann_5.add(Dense(4 ,activation='relu'))
ann_5.add(Dense(4 ,activation='relu'))
#output layer with softmax function
ann_5.add(Dense(5,activation='softmax'))

#compile model
ann_5.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history= ann_5.fit(x_train,y_train, validation_data=(x_val,y_val),epochs=100, batch_size=32)
loss, accuracy= ann_5.evaluate(x_val,y_val)
print('Accuracy:', accuracy)
print('\nLoss:',loss)